# Machine Learning Lifecycle Project

## This is an exercise in 3 parts:
### A. Setup and Message Production
### B. Message Consumption and Storage
### C. Predictive Modeling with Machine Learning
$~$  
# B. Message Consumption and Storage
$~$

# 1. Key Parameters

In [1404]:
#none

# 2. Establish Kafka Consumers

In [1405]:
#import packages
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
import sys, os
import json
from time import sleep
from kafka import KafkaConsumer
from kafka import KafkaProducer
import psycopg2
import pandas.io.sql as psql
%matplotlib inline

In [1406]:
#establish kafka consumers
consumer_RawData = KafkaConsumer('RawData', bootstrap_servers=['localhost:9092'], api_version=(0,10,1), consumer_timeout_ms=1000, group_id='lab2', auto_offset_reset='earliest', enable_auto_commit=True,)
consumer_Results = KafkaConsumer('Results', bootstrap_servers=['localhost:9092'], api_version=(0,10,1), consumer_timeout_ms=1000, group_id='lab2', auto_offset_reset='earliest', enable_auto_commit=True,)

# 3. Consume Data Messages

In [1407]:
#establish PostgreSQL database connection
conn_string = "host=localhost port=5432 dbname=messaging user=postgres password=postgres"
connection=psycopg2.connect(conn_string)
cursor = connection.cursor()
cursor.execute('SELECT version()')
db_version = cursor.fetchone()
print('PostgreSQL database version:',db_version)

PostgreSQL database version: ('PostgreSQL 11.2, compiled by Visual C++ build 1914, 64-bit',)


In [1408]:
#request last record key from PostgreSQL
cursor = connection.cursor()
cursor.execute("select max(key) from RawData")
startkey = cursor.fetchone()
if startkey[0] == None:
    startkey = 0
else:
    startkey = int(startkey[0])
print(startkey,"= last key written to PostgreSQL")

8000 = last key written to PostgreSQL


In [1409]:
#decode json messages and consume with kafka 
RawDataList=[]
ResultsList=[]
    
for message in consumer_RawData:
    decoded_RawData = json.loads(message.value.decode())
    #print(decoded_RawData)
    RawDataList.append(decoded_RawData)
consumer_RawData.close()
print(len(RawDataList),"Raw Data messages consumed from kafka producer")
    
for message in consumer_Results:
    decoded_Results = json.loads(message.value.decode())
    #print(decoded_Results)
    ResultsList.append(decoded_Results)
consumer_Results.close()
print(len(ResultsList),"Result messages consumed from kafka producer")

5000 Raw Data messages consumed from kafka producer
5000 Result messages consumed from kafka producer


# 4. Send Received Data to PostgreSQL

In [1410]:
#append raw data from messages to PostgreSQL
for row in RawDataList:
    cursor.execute(""" INSERT INTO rawdata (key, x, y) VALUES (%s,%s,%s)""", [row['key'],row['x'],row['y']])
    connection.commit()
print (len(RawDataList), "Raw Data records inserted successfully into database")

5000 Raw Data records inserted successfully into database


In [1411]:
#append results from messages to PostgreSQL
for row in ResultsList:
    cursor.execute(""" INSERT INTO results (key, result) VALUES (%s,%s)""", [row['key'],row['result']])
    connection.commit()
print (len(ResultsList), "Result records inserted successfully into database")

5000 Result records inserted successfully into database


In [1412]:
#append predictions key from messages to PostgreSQL
for row in ResultsList:
    cursor.execute("INSERT INTO predictions (key) VALUES (%s)", [row['key']])
    connection.commit()
print (len(ResultsList), "Prediction key records inserted successfully into database")

5000 Prediction key records inserted successfully into database


### PostgreSQL Utility Code

In [1413]:
#close connection
#cursor.close()
#connection.close()
#print("PostgreSQL connection is closed")

In [1414]:
#clear all table records
#cursor = connection.cursor()
#cursor.execute(""" DELETE FROM rawdata""")
#cursor.execute(""" DELETE FROM results""")
#cursor.execute(""" DELETE FROM predictions""")
#connection.commit()

In [1415]:
#rollback on error break
#cursor = connection.cursor()
#cursor.execute(""" rollback;""")